In [ ]:
import logging
# 配置日志
logging.basicConfig(level=logging.INFO)
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Replace 'your_model_file' with the path to the model file you downloaded.
model_id = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_id, device_map="mps", trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(model_id, device_map="mps", trust_remote_code=True, torch_dtype=torch.float16)

In [ ]:
# 构造消息
messages = [
    {"role": "user", "content": "你最喜欢的调料是什么？"},
    {"role": "assistant", "content": "嗯，我特别偏爱新鲜的柠檬汁。它能给我烹饪的食物增添恰到好处的鲜味！"},
    {"role": "user", "content": "你有没有美食的食谱？"}
]

# 应用消息模板
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt")
print(tokenizer.decode(inputs[0]))
inputs = tokenizer(tokenizer.decode(inputs[0]), return_tensors="pt")
inputs.to(model.device)
outputs = model.generate(**inputs, max_new_tokens=4000, do_sample=True, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
# Composing the instruction with the necessary template
instruction = "<s> [INST] {} [/INST]".format("请详细解释一下什么是MOE混合专家大模型的架构和原理，请使用中文回答。")
inputs = tokenizer(instruction, return_tensors="pt")
inputs.to("mps")
outputs = model.generate(**inputs, max_new_tokens=4000, do_sample=True, num_return_sequences=1, temperature=0.9, top_k=50, top_p=0.95, pad_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
import logging
from langchain.llms import HuggingFacePipeline

# Configure logs
logging.basicConfig(level=logging.INFO)

In [ ]:
model_id = "mistralai/Mistral-7B-Instruct-v0.2"
task = "text-generation"  # Specify the task
hf_pipeline = HuggingFacePipeline.from_model_id(model_id, task=task)

In [ ]:
messages = [
    {"role": "user", "content": "What's your favorite spice?"},
    {"role": "assistant", "content": "Well, I have a preference for fresh lemonade. It adds just the right amount of umami to the food I cook!"},
    {"role": "user", "content": "Do you have a recipe for a good meal?"}
]

In [ ]:
from langchain_core.messages.base import BaseMessage
from langchain_core.prompt_values import ChatPromptValue

# Convert messages to a list of BaseMessages
base_messages = [
    BaseMessage(type=message["role"], content=message["content"]) for message in messages
]

# Create a ChatPromptValue from the list of BaseMessages
prompt_value = ChatPromptValue(messages=base_messages)

# Invoke the pipeline with the prompt value
response = hf_pipeline.invoke(prompt_value)
print(response["generated_text"])